In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec


In [ ]:
pc = Pinecone(api_key= os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [1]:
import json
data= json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Very knowledgeable, but his lectures can be a bit dry at times.'},
 {'professor': 'Dr. Jane Doe',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Excellent teacher! Makes complex concepts easy to understand.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 3,
  'review': 'Good professor but the exams are really tough.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Great at engaging students and very approachable.'},
 {'professor': 'Dr. Linda Davis',
  'subject': 'History',
  'stars': 2,
  'review': 'Lectures are not well-organized and hard to follow.'},
 {'professor': 'Dr. Michael Miller',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Good professor with a lot of real-world examples.'},
 {'professor': 'Dr. Karen Wilson',
  'subject': 'English',
  'stars': 3,
  'review': 'Decent lectures, but the grading is quite ha

In [ ]:
processed_data=[]
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
client= genai.GenerativeModel('gemini-1.5-flash')

for review in data['reviews']:
    response= client.generate_content(
        input= review['review'], 
    )